In [13]:
from dataInput import *
import numpy as np
import pandas as pd

In [2]:
solveParamSet = solveParamsInput()

In [3]:
solveParamSet.cutviol

0.0001

In [4]:
intensityFile = 'data/synthetic/intensity.csv';
locationFile = 'data/synthetic/location.csv';
landfallFile = 'data/synthetic/landfall_7.csv';
dissipate_option = 1;

In [5]:
hurricaneDataSet = hurricaneInput(intensityFile, locationFile, landfallFile, dissipate_option)

In [6]:
netNodesFile = 'data/synthetic/nodes.csv';
netParamsFile = 'data/synthetic/netParams.csv';
networkDataSet = networkInput(6,10,0.05,netNodesFile,netParamsFile,hurricaneDataSet)

In [11]:
networkDataSet.cp

array([5.  , 5.25, 5.5 , 5.75, 6.  , 6.25, 6.5 ])

In [14]:
osfname = f"data/synthetic/OOS57.csv"
OS_paths = pd.read_csv(osfname).values  # Read the out-of-sample file

In [49]:
T = 7
S = hurricaneDataSet.states;

# Use a list comprehension and next function
absorbingT = list(OS_paths[3, 0:T]).index(next((x for x in OS_paths[3, 0:T] if S[x][2] == 7 or S[x][0] == 1), None))


In [50]:
absorbingT

2